In [35]:
# importing modules
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
# #from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [68]:
# Reading the data
rheas_tamsat=pd.read_csv('data/rheas/zmb_n_25_tamsat.csv')
# rheas_tamsat=pd.read_csv('data/rheas/maize_zmb_n_25_tamsat_cul1_fertilizer50.csv')
# rheas_chirps=pd.read_csv('data/rheas/maize_zmb_n_25_chirps_cul1_fertilizer50.csv')




In [69]:
# change dates to months or years
# chirps
rheas_tamsat['planting']=pd.to_datetime(rheas_tamsat['planting'], format='ISO8601')
rheas_tamsat['harvest']=pd.to_datetime(rheas_tamsat['harvest'], format='ISO8601')
rheas_tamsat['harvest']=pd.to_datetime(rheas_tamsat['harvest'], format='ISO8601')
rheas_tamsat['year'] = rheas_tamsat['planting'].dt.year
rheas_tamsat['year'] = rheas_tamsat['planting'].dt.year
rheas_tamsat['yield']=rheas_tamsat['gwad']/1000
# rheas_tamsat_raw['harvest']=pd.to_datetime(rheas_tamsat_raw['harvest'], format='%Y/%m/%d')

# chirps
# rheas_chirps['planting']=pd.to_datetime(rheas_chirps['planting'], format='ISO8601')
# rheas_chirps['harvest']=pd.to_datetime(rheas_chirps['harvest'], format='ISO8601')
# rheas_chirps['harvest']=pd.to_datetime(rheas_chirps['harvest'], format='ISO8601')
# rheas_chirps['year'] = rheas_chirps['planting'].dt.year
# rheas_chirps['year'] = rheas_chirps['planting'].dt.year
# rheas_chirps['yield']=rheas_chirps['gwad']/1000

In [70]:
# mask off water bodies eg Lake Victoria
# rheas_tamsat_bimodal=rheas_tamsat_bimodal.loc[rheas_tamsat_bimodal['cname']!='Lake Victoria']


In [71]:
#season [10,11,12, 1,2,3,4,5,6]
#  part1=[1,2, 3, 4,5,6], part1 = [10, 11, 12]
# season part1=[10, 11, 12]
rheas_tamsat_part1=rheas_tamsat
part1 = ['none', 'none', 'none', 'none','none', 'none', 'none', 
                   'none', 'none', 'yes', 'yes', 'yes']
month_to_part1=dict(zip(range(1,13), part1))
rheas_tamsat_part1['season']= rheas_tamsat_part1['planting'].dt.month.map(month_to_part1)

# part2=[1,2, 3, 4,5,6]
rheas_tamsat_part2=rheas_tamsat
part2 = ['yes', 'yes', 'yes', 'yes','yes', 'yes', 'none', 
                   'none', 'none', 'none', 'none', 'none']
unimodal_month_to_part2=dict(zip(range(1,13), part2))
rheas_tamsat_part2['season']= rheas_tamsat_part2['planting'].dt.month.map(unimodal_month_to_part2)

In [72]:
# # Extract data only for the seasons
# long rains (unimodal)
rheas_tamsat_part1= rheas_tamsat_part1.loc[rheas_tamsat_part1['season']=='yes']
rheas_tamsat_part2= rheas_tamsat_part2.loc[rheas_tamsat_part2['season']=='yes']


In [73]:

# extract data for 2022 and 2023 only
rheas_tamsat_2022_2=rheas_tamsat_part1.loc[rheas_tamsat_part1['year']==2022].reset_index()
rheas_tamsat_2023_1=rheas_tamsat_part1.loc[rheas_tamsat_part1['year']==2023].reset_index()
rheas_tamsat_2022=pd.concat([rheas_tamsat_2022_2,rheas_tamsat_2023_1],axis=0)


In [74]:
# # aggregate 
rheas_tamsat_2022=rheas_tamsat_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})

In [75]:
# Read shapefiles and rename columns
country_shapefile = gpd.read_file("data/shp/gadm36_ZMB_0.shp")
district_shp = gpd.read_file("data/shp/gadm36_ZMB_2.shp")

# Rename kenya_shp 
district_shp=district_shp.rename(columns={'NAME1':'cname'})

In [76]:
# Merge data into shapefiles
tamsat_2022 = pd.merge(district_shp, rheas_tamsat_2022, on="cname")

In [77]:
geo_source_tamsat_2022 = GeoJSONDataSource(geojson=tamsat_2022.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [78]:
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p1 = figure(title='Unimodal_Long Rains Season Yield (mt/ha) - 2022',
           outer_width=350, outer_height=300)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source_tamsat_2022,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District: ', '@cname'), ('Yield', '@yield')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [79]:
# color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
# p2 = figure(title='Bimodal_MAM Season Yield (mt/ha) - 2022',
#            outer_width=350, outer_height=300)

# p2.multi_line('xs','ys', source=country_geo_source,
#             line_color='black', line_width=0.5)

# p2.patches('xs','ys', source=geo_source_mam_2022,
#           fill_color={'field': 'yield', 'transform': color_mapper},
#           fill_alpha=0.7, line_color='black', line_width=0.5)

# hover = HoverTool(tooltips=[('District: ', '@cname'), ('Yield', '@yield')])
# p2.add_tools(hover)

# color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
#                      border_line_color=None, location=(0,0), orientation='vertical')
# p2.add_layout(color_bar, 'right')

In [80]:
# # Bimodal Season 2022
# color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
# p3 = figure(title=' Bimodal_ MAM Season  Yield(mt/Ha) - 2023',
#            outer_width=350, outer_height=300)

# p3.multi_line('xs','ys', source=country_geo_source,
#             line_color='black', line_width=0.5)

# p3.patches('xs','ys', source=geo_source_mam_2023,
#           fill_color={'field': 'yield', 'transform': color_mapper1},
#           fill_alpha=0.7, line_color='black', line_width=0.5)

# hover = HoverTool(tooltips=[('District', '@cname'), ('Yield', '@yield')])
# p3.add_tools(hover)

# color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
#                      border_line_color=None, location=(0,0), orientation='vertical')
# p3.add_layout(color_bar, 'right')

In [81]:
# # Bimodal Season 2022
# color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
# p4 = figure(title='Bimodal_OND Season  Yield(mt/ha) - 2022',
#            outer_width=350, outer_height=300)

# p4.multi_line('xs','ys', source=country_geo_source,
#             line_color='black', line_width=0.5)

# p4.patches('xs','ys', source=geo_source_ond_2022,
#           fill_color={'field': 'yield', 'transform': color_mapper1},
#           fill_alpha=0.7, line_color='black', line_width=0.5)

# hover = HoverTool(tooltips=[('District', '@cname'), ('Yield', '@yield')])
# p4.add_tools(hover)

# color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
#                      border_line_color=None, location=(0,0), orientation='vertical')
# p4.add_layout(color_bar, 'right')

In [82]:

# grid=gridplot([ p2, p3, p4, p1],ncols=2, width=650, height=600)
# grid = gridplot([[p2, p3], [None, p4]], width=650, height=600)
grid = gridplot([p1,], ncols=1, width=650, height=600)
show(grid)

In [83]:
from bokeh.io import save
# # # specify the output file name and location
save(grid, filename='../zambia.html', title='Maize Yield Maps')

C:\Users\kasit\AppData\Local\Temp\ipykernel_9576\1141731985.py:3: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(grid, filename='../zambia.html', title='Maize Yield Maps')


'd:\\RONO\\rheas\\maize_crop_modeling\\zambia.html'